In [20]:
from TaskModelling.ResponseFunctions import ResponseFunctions
import os
from Seq2Seq.Encoder import EncoderRNN
from Seq2Seq.Decoder import LuongAttnDecoderRNN
from Seq2Seq.Attention import Attn
import torch
import torch.nn as nn
import unicodedata
import re
from Seq2Seq.GreedySearchDecoder import GreedySearchDecoder
check=torch.cuda.is_available()
from IPython.display import Markdown
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
device=torch.device('cuda' if check else 'cpu')

In [2]:
Task_Path='data/Tasks'
Embedding_Path='C:/Users/mkork/Desktop/BertTry/glove.840B.300d.txt'

In [3]:
functions=ResponseFunctions(Embedding_Path, Task_Path)

In [4]:
save_dir=os.path.join("data", "save")
corpus_name='ConvAIandFriends'
attn_model = 'dot'
model_name = 'ConvAIandFriends'
hidden_size = 600
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64
checkpoint_iter=20000

loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                           '{}_checkpoint.tar'.format(checkpoint_iter))

In [6]:
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    yy = checkpoint['voc_dict']
    xx=checkpoint['embedding']

tuple_list=list(xx.items())
num_words=tuple_list[0][1].shape[0]
embedding = nn.Embedding(num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(xx)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
encoder = encoder.to(device)
decoder = decoder.to(device)

In [7]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder, device)

In [16]:
MAX_LENGTH=10
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc['idx2word'][token.item()] for token in tokens]

    return decoded_words


def startConv(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            res=check_words(input_sentence)
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break

            
            if res:
                if(input_sentence!='bye'):
                    user_i=input_sentence
                    ques=functions.embeddings.check_distance(user_i)
                    quest, response=functions.sort_response(ques,user_i)
                    responsek=response
                    display (Markdown(f'<span style="color: red">Do you want to ask this:{quest}?</span>'))
                else:
                    response=End_conv()
                    Stop=True
                if len(response.split('\n'))>1:
                    for resp_p in response.split('\n'):
                        display(Markdown(f'<span style="color: blue">{resp_p}</span>'))
                else:
                    display(Markdown(f'<span style="color: blue">{response}</span>'))
                        # Normalize sentence
            else:
                input_sentence = sentenceOperation(input_sentence)
                # Evaluate sentence
                output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
                # Format and print response sentence
                output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
                print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [17]:
def check_words(inputs):
    inputs=inputs.split(' ')
    for inputa in inputs:
        if inputa in functions.Related_word:
            return True
    return False

In [18]:
EOS=2
SOS=1
PAD=0
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def indexesFromSentence(voc, sentence):
    return [voc['word2idx'][word] for word in sentence.split(' ')] + [EOS]

def sentenceOperation(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [ ]:
startConv(encoder, decoder, searcher, yy)

> what do we know about corona


<span style="color: red">Do you want to ask this:What animal did 2019-nCoV come from?</span>

<span style="color: blue">The summary of 3500 paper that we know is</span>

<span style="color: blue">  coronavirus (2019-ncov) cause large-sce mortity and morbidity, disrupt trade and travel nworks, and stimulate civil unrest. The 2019-ncOV belongs to bacoronavirus which so contains sars-cov and middle east respiratory syndrome cov (mers-c Cov) The first case of coronav virus disease 2019 was reported in korea on January 20, 2020. The 2019  novel coronavirus (2019-ncov) outbreak now spreading across the world. The pathogen originated  from bats and the intermediate host was the masked pm civ.. editor: la infección por el  nuevo coronav virus ( 2019- NCov) The coronavirus disease 2019 outbreak originated  from wuhan has spread to more than 100 countries. The most likely reservoir  were bats, with evidence that the virus was transmitted to a human through an intermediate host, probably a pm civor raccoon dog. The 2019-ncov is an bacoronavirus that possibly originated from wild anims that genome is highly homologous to bats 2. ivir of a wide range of coronaviruses in vitro and in vivo , may further indicate utility of remdesivir against the novel coronavirus 2019-ncov emerging from wuhan, china. This virus, so originating  from bats, and intermediate hosts, camels and dromedaries, had its greatest expression in the middle east, and the risk of new cases is still not considered to be over. The chinese government has converted 13 large-sce public places in wuhan city, hubei province into makeshift hospits for patients in the coronavirus disease 2019  with mild symptoms. Such conversion will continue to contain the spread of the covid-19. Being "radiologicly idle," at least in the short term, may very well stimulate a desire for meaningful contribution.</span>

> no
Bot: 
> what should we do to protect ourself
Error: Encountered unknown word.
> protection
Bot: what do you do for a living ?
> transmission


...

<span style="color: blue"> The 2019-ncov virus is capable of human-to-human transmission even during its asymptomatic incubation period (3-7 days) The virus has been extremely difficult to contain. It is transmitted by inhaling or contact with infected dropls or fomites, and the incubation. period may range from 2 to 14 days. The virus is transmitted from person to person by respiratory droplers and close contact. Covid-19 is spread by human-to-human transmission through dropl, feco-or, and direct contact and has an incubation period of 2-14 days. Most patients show symptoms within 11.5 days. The median incubation time of covid- 19 has been estimated at approximately 5-6 days, with a range bwee n 1- 14 days. The average incubation period of covid-19 is 5-6 days with the longest incubation Period of 14 days. The virus ranges bwee n 2-14 days (linton ., 2020) and in some cases, transmission is so 47 reported during asymptomatic period.</span>

> mask


<span style="color: red">Do you want to ask this:Does mask useful?</span>

<span style="color: blue">The summary of 3500 paper that we know is</span>

<span style="color: blue"> The lack of agreement on the selection and use of masks  (medicor surgicfacemasks) and respirators (n95/p2/ffp2 or equivent) is reflected in inconsistent and conflicting policies worldwide chughtai ., 2013 ) ..the ccc is a non-profit consortium open to anyone who shares the same vision of making data available to help the public good and ht covid-19. There is no effective and approved vaccine or treatment for covid-19. There is sufficient evidence to recommend stringent hand hygiene and use of face masks. There are no direct scientific evidence regarding wearing protective masks in public spaces for asymptomatic people. The best approach is to develop a broad portfolio of vaccines in a coordinated manner. There are reports of significant shortages of medicstaff, a lack of clinics that can handle and treat infected patients, and high demands  for face masks for protection. There are often no instructions or playbook on how to best proceed in chaotic environments where cause and effect is unclear. stabilizing the system is chlenging. Coronaviruses belong to a family of viruses mainly found in anims but with the recent outbreak , they have transmitted to humans. Study: 37 % of participants admitted a  mask without the apparent signs and symptoms of the infection and more than 75 % felt the need to use sanitizers and gloves. , and, indeed, those who are giving babies formula milk, should "practice respiratory hygiene during feeding," says the WHO. These public heth measures decrease the number of close contacts, resulting in higher degrees of separation within the public,  thereby impeding transmission. However the pandemic has effected the three countries in diversely different pattern hinting at the preparedness of the countries to tackle such naturdisasters.</span>